# Environment setting and package import

In [1]:
import json
import numpy as np
import pandas as pd

from datetime import datetime
from collections import Counter

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%%capture
!pip install wordcloud

In [2]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import BigramAssocMeasures
from nltk.collocations import TrigramCollocationFinder, TrigramAssocMeasures

import re
from PIL import Image

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
%%capture
!pip install accelerate xformers adjustText bitsandbytes

In [6]:
%%capture
!pip install torch torchvision torchaudio

In [7]:
%%capture
!pip install bertopic sentence_transformers

In [10]:
import bitsandbytes
import transformers
from sentence_transformers import SentenceTransformer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration

from umap import UMAP
from hdbscan import HDBSCAN
from torch import bfloat16


from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
from torch import cuda

model_id = 'meta-llama/Llama-2-7b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

print(device)

cuda:0


# Tiktok post import

In [13]:
df = pd.read_csv('/content/drive/MyDrive/social_listening/posts_df.csv')
df

,Unnamed: 0,id,user_name,author_follower,author_heart,location,view,collect,share,digg,comments,desc,cha_desc,all_hashtags,post_date
0,0,7324359490361412869,Carol Brailey|Color Analysis🎨,323700,6800000,[],17444557,177617,11690,2207386,8115,This client’s colour analysis update raises so...,NaN,"['carolbrailey', 'colouranalysis', 'coloranaly...",2024-01-15 17:15:39
1,1,7309534985084521759,marissa ren,2400000,87600000,['Seoul'],15775359,130404,9207,1905913,3221,very fun but humbling experience lol #personal...,NaN,"['personalcoloranalysis', 'coloranalysis', 'ko...",2023-12-06 18:29:06
2,2,7190844699408698667,Rosa,25900,2100000,[],9174808,174663,54000,1483705,3330,me the entire video🤨 #fyp #korea #personalcolo...,NaN,"['fyp', 'korea', 'personalcoloranalysis', 'sou...",2023-01-20 22:10:12
3,3,7130674894198623534,khadija 🍉,4076,1900000,[],12565693,36630,1075,1469326,1008,When I realized I bought two blue tops the day...,NaN,"['personalcolor', 'colortest', 'personalcolora...",2022-08-11 19:40:29
4,4,7164035101205040430,shu ☁️,9355,1700000,[],13440706,42784,3235,1202012,1387,I'm still in denial 🥲 #personalcoloranalysis #...,NaN,"['personalcoloranalysis', 'personalcolor', 'pe...",2022-11-09 16:15:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,469,7162014867740806402,Liana | Color Analysis+Styling,56800,6500000,[],147250,1427,149,3489,51,"Knowing your best colors, prints, accessories,...",NaN,"['fashiontrends', 'coloranalysis', 'personalco...",2022-11-04 05:36:05
470,470,7345820336610397446,Bella ♡,180300,9100000,[],47933,73,6,3037,44,personal color analysis helps too! #personalco...,NaN,"['personalcoloranalysis', 'makeup']",2024-03-13 13:14:40
471,471,7355339480686955782,Wibwub.home.studio,223,4175,[],77928,1613,873,3012,10,เติมความรักให้ตัวเองผ่านศาสตร์ personal color ...,NaN,"['bloomingyoursoul', 'personalcolor', 'persona...",2024-04-08 05:53:50
472,472,7197045049178213637,Carol Brailey|Color Analysis🎨,323700,6800000,[],90393,256,50,3269,91,Colour Analysis Storytime: Featuring a Bright ...,NaN,"['@Carol Brailey|Color Analysis🎨 ', 'carolbrai...",2023-02-06 15:10:35


# Topic modeling

In [14]:
all_sentences = []
for desc in df['desc']:
    # use the sentence tokenizer
    sentences = sent_tokenize(desc)
    # split each post desc into sentences and combine all sentences into a single list
    # all_sentences contains each sentence from every post desc as a separate element
    all_sentences.extend(sentences)

In [15]:
# Pre-calculate embeddings
embedding_model = SentenceTransformer("BAAI/bge-small-en")
embeddings = embedding_model.encode(all_sentences, show_progress_bar=True)
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(2,3)) # use bigram and trigram
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
cluster_model = KMeans(n_clusters=10)

# Pre-reduce embeddings for visualization purposes
reduced_embeddings = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=58).fit_transform(embeddings)


Batches:   0%|          | 0/34 [00:00<?, ?it/s]

In [16]:
# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

# Llama 2 Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# Llama 2 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [17]:
# Our text generator
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=8000,
    repetition_penalty=1.1
)

In [18]:
# System prompt describes information given to all conversations
system_prompt = """
[INST] <>
You are a helpful and honest assistant for finding insights from tiktok posts description.
possible topics may include: makeup and cosmetics, wardrobe, fashion and styling, hair care, cosmetic professionals, cosmetic services...
<>
"""
# Example prompt demonstrating the output we are looking for
example_prompt = """
I have a topic that contains the following documents which are personal color analysis trend descriptions:
- Very fun but humbling experience lol #personalcoloranalysis #coloranalysis #korea
- Exploring personal color analysis and its impact on fashion and style choices. #personalcoloranalysis #fashion #style
- Have you tried personal color analysis? It's a game-changer! #coloranalysis #personalstyle #beauty

The topic is described by the following keywords: 'personal color analysis, color analysis, fashion, style, beauty'.

Based on the information about the topic above, please create a short label of this topic about personal color analysis trend descriptions. Make sure to only return the label and nothing more.

[/INST] Bad customer service with rude attitude.
"""
main_prompt = """
[INST]
I have a topic that contains the following documents which are personal color analysis trend descriptions:
[DOCUMENTS]

TThe topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label of this topic about personal color analysis trend descriptions. Make sure to only return the label and nothing more.
Do not just give a general label such as "personal color analysis trend," but be more specific and highlight the key insights or trends.
[/INST]
"""
prompt = system_prompt + example_prompt + main_prompt

In [19]:
# KeyBERT
keybert = KeyBERTInspired()

# Text generation with Llama 2
llama2 = TextGeneration(generator, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    "Llama2": llama2,
}

topic_model = BERTopic(min_topic_size=4,
                       language="english",
                       calculate_probabilities=True,
                       verbose=True,
                       umap_model=umap_model,
                       hdbscan_model=cluster_model,
                       representation_model=representation_model,
                       embedding_model = 'multi-qa-mpnet-base-dot-v1',
                       vectorizer_model=vectorizer_model)


In [20]:
topics, probs= topic_model.fit_transform(all_sentences)

2024-04-13 19:10:37,746 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

2024-04-13 19:10:53,948 - BERTopic - Embedding - Completed ✓
2024-04-13 19:10:53,950 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-13 19:11:00,167 - BERTopic - Dimensionality - Completed ✓
2024-04-13 19:11:00,169 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-13 19:11:00,369 - BERTopic - Cluster - Completed ✓
2024-04-13 19:11:00,380 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 10/10 [01:50<00:00, 11.07s/it]
2024-04-13 19:12:52,428 - BERTopic - Representation - Completed ✓


In [21]:
result = topic_model.get_topic_info()
result

,Topic,Count,Name,Representation,KeyBERT,Llama2,Representative_Docs
0,0,196,0_colour analysis_color analysis_personal colo...,"[colour analysis, color analysis, personal col...","[colour analysis results, colour analysis upda...",[Personal Color Analysis Trend: Emphasis on Ac...,[The True Light colour analysis client’s colou...
1,1,149,1_personalcolor personalcoloranalysis_personal...,"[personalcolor personalcoloranalysis, personal...","[งต mixandmatchclothes traveletiquette, travel...","[""Personal Color Analysis Trend: Exploring the...",[#ครูกิ๊ฟคริมา #TheReflectionist #TheReflectio...
2,2,145,2_true summer_colour analysis_true winter_deep...,"[true summer, colour analysis, true winter, de...","[summer colour, muted summer, true winter pale...",[Personal Color Analysis Trend: True Summer an...,[I’m fully cool and am a True Summer in colour...
3,3,129,3_personalcolor personalcoloranalysis_koreanma...,"[personalcolor personalcoloranalysis, koreanma...","[personalcolor koreanmakeup kbeauty, personalc...","[K-Beauty Personal Color Analysis Trend, , , ,...",[DM @diod_seoyeon on IG for inquiry 🤍 #persona...
4,4,125,4_colouranalysistok coloranalysistok_personalc...,"[colouranalysistok coloranalysistok, personalc...",[carolbrailey colouranalysistok coloranalysist...,[Personal Color Analysis Trend Description Lab...,[💕🎨⭐️ #carolbrailey #colouranalysistok #colora...
5,5,121,5_let know_difference enhance screen_enhance s...,"[let know, difference enhance screen, enhance ...","[video credit safiyany, video filters makeup, ...","[""Social Media Influencer Shares Personal Colo...",[I enjoy creating my quick videos posted publi...
6,6,91,6_personal color_personalcolor personalcoloran...,"[personal color, personalcolor personalcoloran...","[định personal colors, xác định personal, cách...",[Personal Color Analysis Trend: Exploring Indi...,"[You are you, I am ME!” Nay là bé đẹp Jang Wo..."
7,7,66,7_cocok untuk_di video ini_kita terlihat_ada w...,"[cocok untuk, di video ini, kita terlihat, ada...","[oke dari jumlahnya, kakak di video, note kaka...","[Spring Color Analysis Trend, , , , , , , , , ]","[Tonton videonya dulu sampai abis, dan disini ..."
8,8,36,8_kaospanjangpolos rokpanjangootd_rokpanjangoo...,"[kaospanjangpolos rokpanjangootd, rokpanjangoo...",[outfitkombinasi kaospanjangpolos rokpanjangoo...,[Personal Color Analysis Trend in Indonesia: E...,[💗😆 #personalstyleinspo #personalcoloranalysis...
9,9,7,9____,"[, , , , , , , , , ]","[, , , , , , , , , ]","[Trendy Personal Color Analysis Descriptions, ...","[., ., .]"


In [ ]:
labels = topic_model.get_topics(full=True)

In [ ]:
main_labels = [label[0][0].split("\n")[0] for label in labels["Main"].values()]
main_labels

['colour analysis',
 'personalcolor personalcoloranalysis',
 'true summer',
 'personalcolor personalcoloranalysis',
 'colouranalysistok coloranalysistok',
 'let know',
 'personal color',
 'cocok untuk',
 'kaospanjangpolos rokpanjangootd',
 '']

In [ ]:
bert_labels = [label[0][0].split("\n")[0] for label in labels["KeyBERT"].values()]
bert_labels

['colour analysis results',
 'งต mixandmatchclothes traveletiquette',
 'summer colour',
 'personalcolor koreanmakeup kbeauty',
 'carolbrailey colouranalysistok coloranalysistok',
 'video credit safiyany',
 'định personal colors',
 'oke dari jumlahnya',
 'outfitkombinasi kaospanjangpolos rokpanjangootd',
 '']

In [ ]:
llama2_labels = [label[0][0].split("\n")[0] for label in topic_model.get_topics(full=True)["Llama2"].values()]
llama2_labels

['Personal Color Analysis Trend: Emphasis on Accurate and Consistent Results',
 '"Personal Color Analysis Trend: Exploring the Impact of Personal Colors on Fashion and Style Choices"',
 'Personal Color Analysis Trend: True Summer and Undertone Discovery',
 'K-Beauty Personal Color Analysis Trend',
 'Personal Color Analysis Trend Description Label:',
 '"Social Media Influencer Shares Personal Color Analysis Trends"',
 'Personal Color Analysis Trend: Exploring Individuality Through Skin Tone and Fashion Choices',
 'Spring Color Analysis Trend',
 'Personal Color Analysis Trend in Indonesia: Exploring Kombinations of Outfits and Styles',
 'Trendy Personal Color Analysis Descriptions']

In [22]:
# Visualize the topics
document_list = ["Document " + str(i) for i in range(1, (len(all_sentences)+1))]
topic_model.set_topic_labels(llama2_labels)
topic_model.visualize_documents(document_list, reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=False, custom_labels=True)